<a href="https://colab.research.google.com/github/arara90/Python-Machine-learning/blob/master/ch9_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#텐서플로
* 수치 계산을 위한 오픈소스 소프트웨어 라이브러리로 대규모 머신 러닝에 맞춰 세밀하게 튜닝
* 파이썬으로 계산 그래프 정의 -> 최적화된 C++ 코드 사용해 효율적으로 실행
* 계산 그래프를 여러 부분으로 나누어 여러 CPU or GPU에서 병렬로 실행
* 분산 컴퓨팅 지원


## * 텐서플로 프로그램
1. 구성 : 계산 그래프 만들기
2. 실행 : 그래프 실행 

## 1. 계산 그래프 만들어 세션에서 실행하기

### [구성] 계산 그래프 만들기
* 다음 코드는 실제로 어떠한 계산도 수행하지 않는다. 단지, 계산 그래프만 만든다. (변수 초기화도 X)

In [1]:
import tensorflow as tf
x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')
f= x*x*y + y + 2 

Instructions for updating:
Colocations handled automatically by placer.


### [실행] 계산 그래프 평가 
> **1) ** 세션 시작 
>** 2)** 변수 초기화 
>** 3)** f 평가 (실행)
>**4)** 세션종료

In [5]:
sess = tf.Session()      # 세션시작
sess.run(x.initializer)  # 변수초기화
sess.run(y.initializer)
result = sess.run(f)     # 평가 (실행)
print(result)
 
sess.close()             # 세션종료

42


#### with 사용

In [4]:
with tf.Session() as sess : #with문에서 선언한 세션이 기본 세션
  x.initializer.run()       # tf.get_default_session().run(f)
  y.initializer.run()
  result = f.eval()         #with block이 끝남과 동시에 세션 자동 종료
  
result

42

#### global_variables_initializer() 함수
* 초기화를 바로 수행하지 않고 계산 그래프가 실행될 때 모든 변수를 초기화할 노드 생성

In [0]:
init = tf.global_variables_initializer() #init노드 준비
with tf.Session() as sess : 
  init.run() #실제 모든 변수를 초기화
  result = f.eval()
  
  print(result)

#### InteractiveSession
- 주피터, 파이썬 셸에서 편리
- InteractiveSession이 만들어질 때 자동으로 자신을 기본 세션으로 지정한다. -> with 블록 필요없음.
- 사용이 끝나면 수동으로 종료해줘야 함


In [10]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## 2. 계산 그래프 관리
* 노드를 만들면 자동으로 기본 계산 그래프에 추가

In [12]:

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [13]:
# 독립적인 계산 그래프를 여러 개 만들깨, 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 이를 기본 계산 그래프로 사용한다.
graph = tf.Graph()

with graph.as_default():
  x2 = tf.Variable(2)
  
x2.graph is graph

#tf.reset_default_graph() : 기본 그래프 초기화

True

### 3. 노드 값의 생애주기
* 한 노드를 평가할 때 텐서플로는 이 노드가 의존하고 있는 다른 노드들을 자동으로 찾아 먼저 평가
* 다음 코드에서 y와 z를 평가하기 위해 x를 평가해야 한다는 것을 감지
* 이전에 평가된 w와 x를 재사용 하지 않는다. -> w와 x를 두번 평가함

In [15]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

with tf.Session() as sess:
  print(y.eval()) #10
  print(z.eval()) #15

10
15


* 모든 **노드의 값**은 **계산 그래프 실행 간에 유지 되지 않는다**.
* 예외적으로, **변수**는 그래프 실행간에서 **세션에 의해 유지**된다.   
변수는 초기화될 때 일생이 시작되고, 세션이 종료될 때까지 남아있는다.

#### 한 번의 그래프 실행에서 y와 z를 모두 평가하는 코드

In [16]:
with tf.Session() as sess:
  y_val, z_val = sess.run([y,z])
  print(y_val) #10
  print(z_val) #15


10
15


##3. 텐서플로를 이용한 선형 회귀
* 소스연산 : 상수와 변수 연산은 입력이 없다.
* tensor : 입력과 출력이 되는 다차원 배열
>  * 파이썬 API에서 tensor는 넘파이 ndarray로 나타난다. (텐서를 평가한 결과가 넘파이 배열로 반환)  
>  * 보통은 실수(float)로 채워지지만, 문자열을 저장할수도 있다.  
  

* 아래 코드의 장점은?
>  * numpy를 통해 정규방정식을 직접 계산하는 대신, (GPU버전의 텐서플로를 설치한 후) **GPU**에서 실행한다는 것이다.

In [0]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
#c_ : column추가 

In [0]:
X= tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X" )
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y" )
XT = tf.transpose(X) #전치
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y) #Matmul : 점곱, matrix_inverse : 역행렬

with tf.Session() as sess:
  theta_value = theta.eval()



## 4. 경사하강법 구현
#### 4-1) 직접그래디언트계산
* random_uniform() : 난수를 담은 tensor 생성하는 노드를 그래프에 생성 , 크기와 난수의 범위 입력 ( numpy의 rand() )  
* assign() : 새로운 변수에 새로운 값을 할당하는 노드 생성 
> * 여기서는 배치 경사 하강법의 스텝을 구현
* 반복루프는 훈련단계를 계속 반복해서 실행 (n_epoch만큼), 100번 반복마다 현재의 평균 제곱 에러를 출력 (mse), mse는 매 반복에서 값이 줄어들어야 함.


In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y" )
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name = "theta")
#seed? 

y_pred = tf.matmul(X, theta,name="predictions")
error = y_pred-y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess : 
  sess.run(init)
  for epoch in range(n_epochs):
    if epoch % 100 == 0 :
      print("Epoch : ", epoch, "MSE = " , mse.eval())
    sess.run(training_op)
    
  best_theta = theta.eval()
  



Epoch :  0 MSE =  2.754427
Epoch :  100 MSE =  0.63222194
Epoch :  200 MSE =  0.5727803
Epoch :  300 MSE =  0.5585009
Epoch :  400 MSE =  0.54907006
Epoch :  500 MSE =  0.54228806
Epoch :  600 MSE =  0.5373791
Epoch :  700 MSE =  0.533822
Epoch :  800 MSE =  0.53124255
Epoch :  900 MSE =  0.5293705



#### 4-2) 자동미분 사용
* gradients = 2/m * tf.matmul(tf.transpose(X),error) -> gradients = tf.**gradients**(mse,[theta])[0]
* 입력이 많고 출력이 절을 때 효율적이고 정확한 방법

In [61]:
n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y" )
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name = "theta")
#seed? 

y_pred = tf.matmul(X, theta,name="predictions")
error = y_pred-y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = tf.gradients(mse,[theta])[0]
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess : 
  sess.run(init)
  for epoch in range(n_epochs):
    if epoch % 100 == 0 :
      print("Epoch : ", epoch, "MSE = " , mse.eval())
    sess.run(training_op)
    
  best_theta = theta.eval()
  



Epoch :  0 MSE =  9.882093
Epoch :  100 MSE =  1.0927494
Epoch :  200 MSE =  0.8121932
Epoch :  300 MSE =  0.7291963
Epoch :  400 MSE =  0.6724913
Epoch :  500 MSE =  0.63163954
Epoch :  600 MSE =  0.6021057
Epoch :  700 MSE =  0.5807405
Epoch :  800 MSE =  0.56527746
Epoch :  900 MSE =  0.55408037


#### 4-3) 옵티마이저 사용
* gradients = tf.gradients(mse,[theta])[0]  
training_op = tf.assign(theta, theta - learning_rate*gradients)  
-> opt = tf.train.**GradientDescentOptimizer**(learning_rate=learning_rate)
training_op = opt.minimize(mse)

* 이 외에도 MomentumOptimizer등 다양한 옵티마이저가 있음

In [62]:
n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y" )
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name = "theta")
#seed? 

y_pred = tf.matmul(X, theta,name="predictions")
error = y_pred-y

mse = tf.reduce_mean(tf.square(error), name = "mse")

########

opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = opt.minimize(mse)

#########

init = tf.global_variables_initializer()

with tf.Session() as sess : 
  sess.run(init)
  for epoch in range(n_epochs):
    if epoch % 100 == 0 :
      print("Epoch : ", epoch, "MSE = " , mse.eval())
    sess.run(training_op)
    
  best_theta = theta.eval()

Epoch :  0 MSE =  9.882093
Epoch :  100 MSE =  1.0927494
Epoch :  200 MSE =  0.8121932
Epoch :  300 MSE =  0.7291963
Epoch :  400 MSE =  0.6724913
Epoch :  500 MSE =  0.63163954
Epoch :  600 MSE =  0.60210574
Epoch :  700 MSE =  0.5807405
Epoch :  800 MSE =  0.56527746
Epoch :  900 MSE =  0.55408037


## 5. 훈련 알고리즘에 데이터 주입
* placeholder 노드 : 훈련을 하는 동안 텐서플로에 훈련 데이터를 전달하기 위해 사용
 > * 실제로는 아무런 계산도 하지 않는 특수한 노드, 주입한 데이터를 출력만 한다.
 > *  매 반복마다 X와 y를 다음번 미니 배치로 바꿔야할 때 사용
 > * 출력 텐서의 데이터 타입을 지정, 부가적으로 크기 지정하여 강제할 수 있음. (차원이 None이면 어떤 크기도 가능)
 

In [66]:
A = tf.placeholder(tf.float32, shape=(None,3))
B = A + 5
with tf.Session() as sess:
  B_val_1 = B.eval(feed_dict = {A: [[1,2,3]]})
  B_val_2 = B.eval(feed_dict = {A: [[4,5,6],[7,8,9]]})
  
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [70]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    
    
best_theta

array([[ 2.0699933 ],
       [ 0.8348224 ],
       [ 0.12521975],
       [-0.2431544 ],
       [ 0.3184215 ],
       [ 0.00631773],
       [-0.01373932],
       [-0.8401051 ],
       [-0.8045908 ]], dtype=float32)